In [3]:
import scripts.wall_to_json as s

In [ ]:
def get_dialogs_from_answers():
    from json import JSONDecoder
    j = JSONDecoder()
    
    already_used_id = []
    dialogs = []

    answers = source_answers.copy()
    anchor = answers[list(map(lambda x: x!='[]', answers['parents_stack']))].head(1)
    while not anchor.empty:
        dialog = []
        already_used_id.append(int(anchor['comment_id']))

        parent_id = j.decode(answers['parents_stack'].iloc[0])[0]
        dialog.append(source_comments[source_comments['comment_id'] == parent_id].head(1))
        dialog.append(anchor)

        after_anchor = answers[answers['reply_to_comment'] == int(anchor['comment_id'])].head(1)
        while not after_anchor.empty:
            anchor = after_anchor
            already_used_id.append(anchor['comment_id'])
            dialog.append(anchor)
            after_anchor = answers[answers['reply_to_comment'] == int(anchor['comment_id'])].head(1)

        if len(pd.concat(dialog)) > 1:
            dialogs.append(pd.concat(dialog))

        answers = answers[~answers['comment_id'].isin(already_used_id)]
        anchor = answers[list(map(bool, answers['parents_stack']))].head(1)
    return dialogs